In [ ]:
%%capture
!pip install sentencepiece
!pip install datasets
!pip install py7zr
!pip install torch
!pip install sacrebleu
!pip install rouge_score
!pip install transformers[torch]

In [ ]:
from datasets import load_dataset
import torch
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm


In [ ]:
#dataset_samsum = load_dataset('csv', data_files= '/content/news_headlines_1.csv',split='validation') #engine='python', quoting=csv.QUOTE_NONE)
data_files = {"train": "/content/drive/MyDrive/Otherweb/seo_short_headline/seo_headline_train.csv",
              "valid": "/content/drive/MyDrive/Otherweb/seo_short_headline/seo_headline_valid.csv",
              "test": "/content/drive/MyDrive/Otherweb/seo_short_headline/seo_headline_test.csv"}
dataset_samsum = load_dataset('csv', data_files=data_files)
dataset_samsum

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Excerpt', 'Summary'],
        num_rows: 49673
    })
    valid: Dataset({
        features: ['Excerpt', 'Summary'],
        num_rows: 975
    })
    test: Dataset({
        features: ['Excerpt', 'Summary'],
        num_rows: 96
    })
})

In [ ]:
#df = pd.read_csv('/content/news_headlines_1.csv') #engine='python', quoting=csv.QUOTE_NONE, header=None,delimiter=",")

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'google/pegasus-cnn_dailymail'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = PegasusTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer.batch_encode_plus(example_batch["Excerpt"],
                                                pad_to_max_length=True, max_length=200, truncation=True)
  target_encodings = tokenizer.batch_encode_plus(example_batch["Summary"],
                                                pad_to_max_length=True, max_length=20, truncation=True)
  return {"input_ids": input_encodings["input_ids"],
          "attention_mask": input_encodings["attention_mask"],
          "labels": target_encodings["input_ids"]}
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features,
                                        batched=True)

columns = ["input_ids", "labels", "attention_mask"]
dataset_samsum_pt.set_format(type="torch", columns=columns)

Map:   0%|          | 0/49673 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/975 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='seo-headline_2', num_train_epochs=6, warmup_steps=500,
    per_device_train_batch_size=8, per_device_eval_batch_size=8,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16)

In [ ]:
#hide_output
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["valid"])

In [ ]:
trainer.train()


Step,Training Loss,Validation Loss
500,0.803100,0.714160
1000,0.611700,0.594798
1500,0.556800,0.575545
2000,0.521900,0.568170


TrainOutput(global_step=2328, training_loss=1.7527133853165144, metrics={'train_runtime': 10083.3689, 'train_samples_per_second': 29.557, 'train_steps_per_second': 0.231, 'total_flos': 1.681470767087616e+17, 'train_loss': 1.7527133853165144, 'epoch': 6.0})

In [ ]:
from datasets import load_metric

bleu_metric = load_metric("sacrebleu")
rouge_metric = load_metric("rouge")

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

<ipython-input-12-3e48a45e089f>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def evaluate_summaries_pegasus(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="Excerpt",
                               column_summary="Summary"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=200,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=20)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score

In [ ]:
score = evaluate_summaries_pegasus(
    dataset_samsum["test"], rouge_metric, trainer.model, tokenizer,
    batch_size=2, column_text="Excerpt", column_summary="Summary")

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"pegasus"])

  0%|          | 0/48 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1298: UserWarning: Unfeasible length constraints: `min_length` (32) is larger than the maximum possible length (20). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(
100%|██████████| 48/48 [00:23<00:00,  2.07it/s]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.383108,0.167876,0.355946,0.356255


In [ ]:
trainer.push_to_hub("Training complete!")

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

events.out.tfevents.1707228623.adb2d82b8c1a.1198.0:   0%|          | 0.00/42.7k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abdulmatinomotoso/seo-headline_2/commit/e68caf4877591790341ce188d8668221e68a1a9c', commit_message='Training complete!', commit_description='', oid='e68caf4877591790341ce188d8668221e68a1a9c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# hide
import transformers
from transformers import pipeline
transformers.logging.set_verbosity_error()

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
sample_text = dataset_samsum["test"][0]["article"]
reference = dataset_samsum["test"][0]["headline"]
pipe = pipeline("summarization", model="abdulmatinomotoso/pegasus-samsum")

print("Dialogue:")
print(sample_text)
print("\nReference Summary:")
print(reference)
print("\nModel Summary:")


Dialogue:
Moscow-born Elena Rybakina became Kazakhstan's first ever winner of Wimbledon today - despite a ban on Russian athletes.The tennis star, 23, who still lives in the Russian capital, has managed to dodge the ban by switching her allegiance and faced Tunisia's Ons Jabeur in Saturday's final on Centre Court.Rybakina rallied from a set down to deny Jabeur her own slice of history with a 3-6 6-2 6-2 victory over the Tunisian world No.2 in Saturday's showdown at the All England Club.The 23-year-old Rybakina is also the youngest women's Wimbledon champion since Petra Kvitova in 2011 after reducing Jabeur to a frustrated wreck during the one-hour, 48-minute title decider.Her semi-final victory had been cheered by Russian-media - 'Russian-born star storms into Wimbledon final,' cheered the front page of the Russia Today website.However four years ago Rybakina decided to represent Kazakhstan in tournaments - and she appears to be fed up of questions about Russia.Speaking after her sensa

In [ ]:
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

IndexError: ignored

In [ ]:
custom_dialogue = """\
The NAACP is calling on the Boston School Committee not to choose a new superintendent Wednesday, calling the search process “fundamentally flawed” from its short timeline to the lack of Black and Latino finalists.“The lack of representation in the finalist pool should have immediately caused the process to pause, review, and reopen (if necessary),” Tanisha Sullivan, the organization’s president, wrote in a letter that was sent to School Committee Chair Jeri Robinson Friday.A search committee, jointly appointed by Mayor Michelle Wu and the School Committee, had hoped to present a more diverse slate of finalists, but two would be-finalists, a Latina and a Black woman, withdrew prior to the announcement. The remaining two are Somerville Superintendent Mary Skipper, who is white, and Tommy Welch, who is Japanese American and white.They are competing to replace outgoing Superintendent Brenda Cassellius who is leaving Thursday.Meanwhile, competition for Skipper intensified Monday night. The School Committee authorized its chair to sign a two-year extension to Skipper’s contract, which expires Thursday. The agreement, however, still allows for Skipper to go to Boston with at least 90 days notice.Having no Black or Latino finalists is a departure from past superintendent searches in a district where about three-quarters of students identify that way. A number of education advocates, including City Councilor Julia Mejia, have expressed disappointment over the lack of Black and Latino finalists.Sullivan in her letter urged the School Committee to expand the slate of finalists so there’s a more robust mix of representation and experience, noting that having only “two finalists for a nationally respected district like Boston should raise an automatic caution flag in the process.”The search also was done at an unusually fast pace, Sullivan said. The job was posted in April and finalists were chosen two months later. Previous searches spent many months between those two critical bookends recruiting and vetting candidates.Other organizations, including Democrats for Education Reform and Bostonians for an Elected School Committee, also have raised concerns about the short timeline and wonder if a longer search would have yielded a more robust final slate.Robinson immediately rejected the NAACP’s request to expand the pool of finalists, according to a letter she sent Friday to Sullivan, which was shared with the Globe, noting “our kids cannot wait any longer, and we cannot let this opportunity pass us by.”“While I share your disappointment that our search did not result in a more racially diverse group of public interviews, I stand by this process. I am confident both candidates have the necessary qualifications and are uniquely positioned to lead our District forward,” Robinson wrote.Sullivan, who is running for secretary of state, also questions whether the search could have yielded a larger pool of finalists if it waited until after Wu and state Education Commissioner Jeffrey Riley finished negotiating a district-improvement plan, which was completed Monday.Sullivan stressed that she thinks highly of both finalists and noted both are qualified to be superintendent, but added the public deserves to have a more robust slate. Wu told the Globe last week looming concerns about a possible state takeover of Boston Public Schools weighed on potential candidates.Sullivan reiterated her concerns in an interview Tuesday with the Globe.“I do think the process itself yielded the results we have because the district underestimated the impact that the state level issues would have on otherwise interested candidates being willing to move forward,” Sullivan said. “To me having this agreement signed by the interested parties now helps to set fully both what the educational and political landscapes will be for anyone coming into the district.”The Globe’s Great Divide team explores educational inequality in Boston and statewide. Sign up to receive our newsletter, and send ideas and tips to thegreatdivide@globe.com.James Vaznis can be reached at james.vaznis@globe.com. Follow him on Twitter @globevaznis.",
"""
print(pipe(custom_dialogue, **gen_kwargs)[0]["summary_text"])

The NAACP is calling on the Boston School Committee not to choose a new superintendent Wednesday .<n>A search committee had hoped to present a more diverse slate of finalists, but two would be-finalists, a Latina and a Black woman, withdrew prior to the announcement .<n>The remaining two are Somerville Superintendent Mary Skipper, who is white, and Tommy Welch, who is Japanese American and white .
